### 사용자 정의 데이터셋과 모델과 학습
- iris.csv ==> 사용자 정의 데이터셋
- DNN 모델 ==> 사용자 정의 모델


[1] 모듈 로딩 및 데이터 준비 <hr>

In [59]:
# 모듈로딩
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchmetrics.classification import F1Score
from torchinfo import summary

from torch.utils.data import Dataset,DataLoader

import pandas as pd
from sklearn.preprocessing import LabelEncoder      # 타겟컬럼 수치화

In [60]:
# 데이터
DATA_FILE='../DATA/iris.csv'

In [61]:
# CSV => DataFrame
irisDF=pd.read_csv(DATA_FILE)
irisDF.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [62]:
# 타겟컬럼 수치화 ==> LabelEncoder
encoder=LabelEncoder()
encoder.fit(irisDF['variety'])
irisDF['variety']=encoder.transform(irisDF['variety'])

In [63]:
irisDF.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


[2] 사용자 정의 데이터셋 클래스 생성 <hr>

---
- 클래스목적 : 학습용 데이터셋 텐서화 및 전처리
- 클래스이름 : CustomDataSet
- 부모클래스 : torch.utils.data.Dataset
- 매개  변수 : featureDF ,targetDF
--- 



In [64]:
class CustomDataset(Dataset):
    # 데이터 로딩 및 전처리 진행과 인스턴스 생성 메서드
    def __init__(self,featureDF,targetDF):
        super().__init__()
        self.featureDF=featureDF
        self.targetDF=targetDF
        self.n_rows=featureDF.shape[0]
        self.n_features=featureDF.shape[1]
    
    # 데이터의 개수 반환 메서드
    def __len__(self):
        return self.n_rows

    # 특정 idx의 데이터와 타겟 반환 메서드 => Tensor반환!
    def __getitem__(self,idx):
        featureTS=torch.FloatTensor(self.featureDF.iloc[idx].values)
        targetTS=torch.FloatTensor(self.targetDF.iloc[idx].values)
        return featureTS,targetTS

---
- 함수기능 : 파일 확장자별 데이터 프레임 변환 기능
- 함수이름 : convertDataFrame
- 매개변수 : data_path
- 함수결과 : DataFramee
---

In [65]:
def convertDataFrame(data_path,exist_header=0):
    ext=data_path.rsplit('.')[-1]  # 파일 확장자
    if ext=='csv':
        return pd.read_csv(data_path, header=exist_header)
    elif ext=='json':
        return pd.read_json(data_path, header=exist_header)
    elif ext in ['xlsx','xls']:
        return pd.read_excel(data_path, header=exist_header)
    else:
        return pd.read_table(data_path, header=exist_header)

---
- 클래스기능 : 파일기반 데이터셋
- 클래스이름 : FileDataset
- 매개 변수 : data_path 파일경로
- 부모클래스 : util.data.Dataset
---

In [66]:
class FILEDataset(Dataset):
    # 데이터 로딩 및 전처리 진행과 인스턴스 생성 메서드
    def __init__(self,data_path):
        super().__init__()
        dataDF=convertDataFrame(data_path)
        self.featureDF=dataDF[dataDF.columns[:-1]]
        self.targetDF=dataDF[dataDF.columns[[-1]]]
        self.n_features=self.featureDF.shape[0]
        self.n_rows=self.featureDF.shape[1]
    
    # 데이터의 개수 반환 메서드
    def __len__(self):
        return self.n_rows

    # 특정 idx의 데이터와 타겟 반환 메서드 => Tensor반환!
    def __getitem__(self,idx):
        featureTS=torch.FloatTensor(self.featureDF.iloc[idx].values)
        targetTS=torch.FloatTensor(self.targetDF.iloc[idx].values)
        return featureTS,targetTS

[3] 데이터셋 인스턴스 생성<hr>

In [67]:
# 피쳐와 타겟 추출
featureDF=irisDF[irisDF.columns[:-1]]
targetDF=irisDF[irisDF.columns[[-1]]]
print(f'featureDF : {featureDF.shape}, targetDF : {targetDF.shape}')

featureDF : (150, 4), targetDF : (150, 1)


In [68]:
# IRIS 데이터셋 인스턴스 생성
irisDS=CustomDataset(featureDF,targetDF)

In [69]:
# IRIS 데이터셋 속성
irisDS.n_features, irisDS.n_rows

(4, 150)

In [70]:
# IRIS 데이터셋 메서드
irisDS[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0.]))

[4] 데이터로더 인스턴스 생성

In [71]:
## 필요한 것 : Dataset 인스턴스, (Default) Batch_size = 1
irisDL=DataLoader(irisDS,batch_size=5)

In [72]:
for dataTS,targetTS in irisDL:
    print(dataTS.shape,targetTS.shape)
    break

torch.Size([5, 4]) torch.Size([5, 1])


[5] 모델준비 <hr>

--------------------------------------------------------------------------------------------------------
- 모델이름 : CustomModel
- 부모클래스 : nn.Module
- 매개변수 : in_in, out_out, h_ins=[ ]
- 모델구조
    - 입력층 : 입력 4개  / 출력 n개 / AF ReLU -> LeakyReLU
    - 은닉층 : 입력 n개 /출력 n개 /AF ReLU -> LeakyReLU
    - 출력층 :  입력 n개 /   출력 3개      /  AF 분류 - 다중 softmax
--------------------------------------------------------------------------------------------------------

In [73]:
class CustomModel(nn.Module):
    # 모델 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()
        self.in_layer=nn.Linear(4,10)
        self.h_layer=nn.Linear(10,30)
        self.out_layer=nn.Linear(30,3)

    # 순방향 학습 메서드
    def forward(self,x):
        y=F.relu(self.in_layer(x))
        y=F.relu(self.h_layer(y))
        # F.softmax(self.out_layer(y))


In [74]:
model = CustomModel()
print(model)
summary(model)

CustomModel(
  (in_layer): Linear(in_features=4, out_features=10, bias=True)
  (h_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=3, bias=True)
)


Layer (type:depth-idx)                   Param #
CustomModel                              --
├─Linear: 1-1                            50
├─Linear: 1-2                            330
├─Linear: 1-3                            93
Total params: 473
Trainable params: 473
Non-trainable params: 0

In [75]:
model = CustomModel()
print(model)
summary(model, input_size=(100000,4))

CustomModel(
  (in_layer): Linear(in_features=4, out_features=10, bias=True)
  (h_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=3, bias=True)
)


Layer (type:depth-idx)                   Output Shape              Param #
CustomModel                              --                        93
├─Linear: 1-1                            [100000, 10]              50
├─Linear: 1-2                            [100000, 30]              330
Total params: 473
Trainable params: 473
Non-trainable params: 0
Total mult-adds (M): 38
Input size (MB): 1.60
Forward/backward pass size (MB): 32.00
Params size (MB): 0.00
Estimated Total Size (MB): 33.60

[6] 학습 <hr>

In [78]:
## 배치 크기만큼 데이터와 타겟 추출해서 학습 진행 

ts_loss, ts_score = [[],[]], [[],[] ]
for epoch in range(10):
    
    loss_total, score_totla = 0,0
    for dataTS, targetTS in irisDL:

        print(dataTS.shape, targetTS.shape, targetTS.dtype)
        targetTS = targetTS.reshape(-1).long()
        
        # 배치 크기 만큼 학습 진행 
        pre_y = model(dataTS)
        print(pre_y.shape, targetTS.reshape(-1).shape)

        # # 손실 계산 
        # loss = nn.CrossEntropyLoss()(pre_y, targetTS)

        break
    

torch.Size([5, 4]) torch.Size([5, 1]) torch.float32


AttributeError: 'NoneType' object has no attribute 'shape'